In [ ]:
import pandas as pd
import plotly.graph_objects as go
import os
import random
from datetime import *
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

sender_email = "danarms.2421@gmail.com"
receiver_emails = ["danarms.2421@gmail.com", "falbar.2421@gmail.com"]
password = input("Type your password and press enter:")

message = MIMEMultipart("alternative")
message["Subject"] = "Live Partners and On-Boarded Partners Geomap Report"
message["From"] = sender_email
message["To"] = ", ".join(receiver_emails)

df = pd.read_csv('C:\\Users\\saima\\OneDrive\\Desktop\\Book1.csv')
print(df)
for col in df.columns:
    df[col] = df[col].astype(str)
    
df['text'] = df['COUNTRY'] + '<br>' + \
    'ONBOARDED PARTNERS: ' + df['ONBOARDED PARTNERS']

fig = go.Figure(data=go.Choropleth(
    locations = df['CODE'],
    z = df['LIVE PARTNERS'].astype(float),
    text = df['text'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Live Partners',
))

fig.update_layout(
    title_text='Live Partners and On-Boarded Partners',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='MyCompany.com',
        showarrow = False
    )]
)

if not os.path.exists("images"):
    os.mkdir("images")
    
fig.write_image("images/LiveOnboardedPartners.png")

attachment = 'images/LiveOnboardedPartners.png'

fp = open(attachment, 'rb')
img = MIMEImage(fp.read())
fp.close()
img.add_header('Content-ID', '<{}>'.format(attachment))

html = """\
    <html>
     <body>
        <p>Hi!
            <br>Hope your are doing well
            <br>
            <br>Here is the Live Partners and On-Boarded Partners Geomap Report<br>
        </p>
        <p><img src="cid:images/LiveOnboardedPartners.png" alt="Sales Scatter Graph"></p>
      </body>
    </html>"""

msg = MIMEText(html, "html")
message.attach(msg)
message.attach(img)
context = ssl.create_default_context()

try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_emails, message.as_string())
        print ("Email Sent Successfully!")
except SMTPException:
    print ("Error: Unable to Send Email.")